# Exercise 8-1: Analyze the Forest Fires data

## Read the data

In [3]:
import pandas as pd

In [4]:
fires_by_month = pd.read_pickle('fires_by_month.pkl')

In [5]:
# display the first five rows
fires_by_month.head()

,state,fire_year,fire_month,acres_burned,days_burning,fire_count
0,AK,1992,5,4202.0,135.0,14
1,AK,1992,6,86401.0,417.0,23
2,AK,1992,7,48516.7,500.0,26
3,AK,1992,8,3305.0,92.0,4
4,AK,1992,9,20.0,1.0,1


## Group and aggregate the data

In [6]:
# group the data by state and year
fires_grouped = fires_by_month.groupby(["state", "fire_year"])
fires_grouped.head()

C:\Users\s225146371\AppData\Local\Temp\ipykernel_14180\2630722329.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fires_grouped = fires_by_month.groupby(["state", "fire_year"])


,state,fire_year,fire_month,acres_burned,days_burning,fire_count
0,AK,1992,5,4202.0,135.0,14
1,AK,1992,6,86401.0,417.0,23
2,AK,1992,7,48516.7,500.0,26
3,AK,1992,8,3305.0,92.0,4
4,AK,1992,9,20.0,1.0,1
...,...,...,...,...,...,...
9289,WY,2015,2,1420.0,1.0,2
9290,WY,2015,3,2484.7,7.0,10
9291,WY,2015,4,802.7,7.0,6
9292,WY,2015,5,121.0,2.0,2


In [7]:
# sum the grouped data and store DataFrame in fires_by_year variable
fires_by_year = fires_grouped.sum()
fires_by_year.head(10)

fire_month  acres_burned  days_burning  fire_count
state fire_year                                                    
AK    1992               35  1.424447e+05        1145.0          68
      1993               30  6.866305e+05        3373.0         144
      1994               48  2.616047e+05        2517.0         126
      1995               41  4.376260e+04         880.0          48
      1996               40  5.984072e+05        2112.0          88
      1997               33  2.025818e+06        2295.0         128
      1998               30  1.206034e+05         435.0          31
      1999               31  1.005597e+06        2312.0         101
      2000               42  7.561614e+05        1745.0          51
      2001               37  2.197077e+05        1659.0          26

In [8]:
# drop the fire_month column
fires_by_year.drop(columns="fire_month", inplace=True)
fires_by_year.head()

acres_burned  days_burning  fire_count
state fire_year                                        
AK    1992           142444.7        1145.0          68
      1993           686630.5        3373.0         144
      1994           261604.7        2517.0         126
      1995            43762.6         880.0          48
      1996           598407.2        2112.0          88

## Use pivot tables

In [9]:
# select rows where the year is 2013 or later
fires_recent = fires_by_year.query('fire_year >= 2013').reset_index()
fires_recent.head()

,state,fire_year,acres_burned,days_burning,fire_count
0,AK,2013,1320518.50,4644.0,153
1,AK,2014,233426.40,571.0,43
2,AK,2015,5136372.99,16636.0,340
3,AL,2013,28015.65,115.0,581
4,AL,2014,35310.20,107.0,758


In [10]:
# use the pivot() method to pivot the data so the state values are used as row labels, 
# fire_year values are used as column labels, and acres_burned values are used as data
fires_recent.pivot(index='state', columns='fire_year', values='acres_burned').head()

fire_year,2013,2014,2015
state,,,
AK,1320518.50,233426.40,5136372.99
AL,28015.65,35310.20,28386.96
AR,11797.60,18521.80,16302.96
AZ,102307.47,202212.51,154720.38
CA,575555.65,543033.33,844674.02


In [11]:
# use the pivot_table() method with the fires_by_month DataFrame to get the same result
# as the previous cell
fires_by_month.query('fire_year >= 2013').reset_index().pivot_table(index="state", columns="fire_year", values="acres_burned", aggfunc='sum').head()


C:\Users\s225146371\AppData\Local\Temp\ipykernel_14180\4007455134.py:3: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  fires_by_month.query('fire_year >= 2013').reset_index().pivot_table(index="state", columns="fire_year", values="acres_burned", aggfunc='sum').head()


fire_year,2013,2014,2015
state,,,
AK,1320518.50,233426.40,5136372.99
AL,28015.65,35310.20,28386.96
AR,11797.60,18521.80,16302.96
AZ,102307.47,202212.51,154720.38
CA,575555.65,543033.33,844674.02


## Work with bins

In [12]:
# reset the index for the DataFrame named fires_by_year
fires_by_year.reset_index(inplace=True)
fires_by_year

,state,fire_year,acres_burned,days_burning,fire_count
0,AK,1992,142444.70,1145.0,68
1,AK,1993,686630.50,3373.0,144
2,AK,1994,261604.70,2517.0,126
3,AK,1995,43762.60,880.0,48
4,AK,1996,598407.20,2112.0,88
...,...,...,...,...,...
1219,WY,2011,134336.25,669.0,160
1220,WY,2012,481206.46,1802.0,215
1221,WY,2013,53884.90,742.0,40
1222,WY,2014,7179.43,224.0,48


In [13]:
# bin the data by decade
fires_by_year['decade'] = pd.cut(fires_by_year.fire_year, bins=[1990,2000,2010,2020], labels=['1990s', '2000s', '2010s'], right=False)
fires_by_year.head(10)

,state,fire_year,acres_burned,days_burning,fire_count,decade
0,AK,1992,1.424447e+05,1145.0,68,1990s
1,AK,1993,6.866305e+05,3373.0,144,1990s
2,AK,1994,2.616047e+05,2517.0,126,1990s
3,AK,1995,4.376260e+04,880.0,48,1990s
4,AK,1996,5.984072e+05,2112.0,88,1990s
5,AK,1997,2.025818e+06,2295.0,128,1990s
6,AK,1998,1.206034e+05,435.0,31,1990s
7,AK,1999,1.005597e+06,2312.0,101,1990s
8,AK,2000,7.561614e+05,1745.0,51,2000s
9,AK,2001,2.197077e+05,1659.0,26,2000s


In [14]:
# double-check the edge values
fires_by_year.head()

,state,fire_year,acres_burned,days_burning,fire_count,decade
0,AK,1992,142444.7,1145.0,68,1990s
1,AK,1993,686630.5,3373.0,144,1990s
2,AK,1994,261604.7,2517.0,126,1990s
3,AK,1995,43762.6,880.0,48,1990s
4,AK,1996,598407.2,2112.0,88,1990s


In [15]:
# drop the fire_year column and assign the DataFrame that's returned to a variable named fires_by_decade
fires_by_decade = fires_by_year.drop(columns=['fire_year'])
fires_by_decade.head()

,state,acres_burned,days_burning,fire_count,decade
0,AK,142444.7,1145.0,68,1990s
1,AK,686630.5,3373.0,144,1990s
2,AK,261604.7,2517.0,126,1990s
3,AK,43762.6,880.0,48,1990s
4,AK,598407.2,2112.0,88,1990s


In [17]:
# group by the state and decade columns and sum the data
fires_by_decade = fires_by_decade.groupby(['state', 'decade']).sum()
fires_by_decade.head()

C:\Users\s225146371\AppData\Local\Temp\ipykernel_14180\1563968234.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fires_by_decade = fires_by_decade.groupby(['state', 'decade']).sum()


acres_burned  days_burning  fire_count
state decade                                        
AK    1990s   4.884868e+06       15069.0         734
      2000s   1.892062e+07       34174.0         974
      2010s   8.420518e+06       31025.0         915
AL    1990s   1.945928e+05         214.0         216
      2000s   4.088805e+05        1863.0         535